In [1]:
# Import of every needed library
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from multiprocessing import Pool
import tensorflow as tf
import pandas as pd
import numpy as np
import random
import pickle
import h5py
import gzip
import time
import csv
import sys
import os

In [2]:
def createFolders(model_name, save_model_path):
    # Iterates over all existing models and chooses the right folder to save everything 
    file_paths = os.listdir(save_model_path)
    for path in file_paths:
        name = '_' + model_name
        if path.endswith(name):
            correct_path = path 

    # Creates missing folders or chooses the right one to append new data to
    if 'correct_path' in locals():
        folder_path = os.path.join(save_model_path, correct_path)
    else:
        folder_number = len(os.listdir(save_model_path))+1
        folder_path = save_model_path + '/' + str(folder_number) + '_' + model_name
        os.mkdir(folder_path)

        # Creates the csv to save every models performance in
        c_count = model_name.count('c')
        depth_names = []
        for i in range(c_count):
            depth_names.append('Depth_{}'.format(i+1))
        columns = ['Learning_Rate','Batch_Size','Patch_Size']
        columns.extend(depth_names)
        columns.extend(['Hidden_Nodes','Accuracy','Auc','Steps', 'Early_Stopped','Time', 'Title'])

        with open(os.path.join(folder_path, model_name+'_Hyperparameter.csv'), 'a') as f:
            writer = csv.writer(f)
            writer.writerow(columns)
            
    return folder_path

In [3]:
def metaYielder(path_mc_images):
    with h5py.File(path_mc_images, 'r') as f:
        keys = list(f.keys())
        events = []
        for key in keys:
            events.append(len(f[key]))
            
    gamma_anteil = events[0]/np.sum(events)
    hadron_anteil = events[1]/np.sum(events)
    
    gamma_count = int(round(num_events*gamma_anteil))
    hadron_count = int(round(num_events*hadron_anteil))
    
    return gamma_anteil, hadron_anteil, gamma_count, hadron_count

In [4]:
def batchYielder():
    gamma_anteil, hadron_anteil, gamma_count, hadron_count = metaYielder()

    gamma_batch_size = int(round(batch_size*gamma_anteil))
    hadron_batch_size = int(round(batch_size*hadron_anteil))

    for step in range(num_steps):
        gamma_offset = (step * gamma_batch_size) % (gamma_count - gamma_batch_size)
        hadron_offset = (step * hadron_batch_size) % (hadron_count - hadron_batch_size)

        with h5py.File(path_mc_images, 'r') as f:
            gamma_data = f['Gamma'][gamma_offset:(gamma_offset + gamma_batch_size), :, :, :]
            hadron_data = f['Hadron'][hadron_offset:(hadron_offset + hadron_batch_size), :, :, :]

        batch_data = np.concatenate((gamma_data, hadron_data), axis=0)
        labels = np.array([True]*gamma_batch_size+[False]*hadron_batch_size)
        batch_labels = (np.arange(2) == labels[:,None]).astype(np.float32)

        yield batch_data, batch_labels

In [5]:
def getValidationTesting(path_mc_images, events_in_validation_and_testing, gamma_anteil, hadron_anteil, gamma_count, hadron_count):
    with h5py.File(path_mc_images, 'r') as f:
        gamma_size = int(round(events_in_validation_and_testing*gamma_anteil))
        hadron_size = int(round(events_in_validation_and_testing*hadron_anteil))

        gamma_valid_data = f['Gamma'][gamma_count:(gamma_count+gamma_size), :, :, :]
        hadron_valid_data = f['Hadron'][hadron_count:(hadron_count+hadron_size), :, :, :]

        valid_dataset = np.concatenate((gamma_valid_data, hadron_valid_data), axis=0)
        labels = np.array([True]*gamma_size+[False]*hadron_size)
        valid_labels = (np.arange(2) == labels[:,None]).astype(np.float32)


        gamma_test_data = f['Gamma'][(gamma_count+gamma_size):(gamma_count+2*gamma_size), :, :, :]
        hadron_test_data = f['Hadron'][(hadron_count+hadron_size):(hadron_count+2*hadron_size), :, :, :]

        test_dataset = np.concatenate((gamma_test_data, hadron_test_data), axis=0)
        labels = np.array([True]*gamma_size+[False]*hadron_size)
        test_labels = (np.arange(2) == labels[:,None]).astype(np.float32)
        
    return valid_dataset, valid_labels, test_dataset, test_labels

In [6]:
def bestAuc(folder_path, architecture):
    # Loading the existing runs to find the best auc untill now. Only a model with a better auc will be saved
    df = pd.read_csv(os.path.join(folder_path, architecture+'_Hyperparameter.csv'))
    if len(df['Auc']) > 0:
        best_auc = df['Auc'].max()
    else:
        best_auc = 0
        
    return best_auc

In [7]:
def getHyperparameter(architecture, number_of_nets):
    # Hyperparameter for the model (fit manually)
    num_labels = 2 # gamma or proton
    num_channels = 1 # it is a greyscale image
    
    num_steps = 20001     # Maximum batches for the model
    
    min_batch_size = 64   # How many images will be in a batch
    max_batch_size = 257
    
    patch_size = [3, 5]   # Will the kernel/patch be 3x3 or 5x5

    min_depth = 2         # Setting the depth of the convolution layers. New layers will be longer than the preceding
    max_depth = 21
    
    min_num_hidden = 8    # Number of hidden nodes in f-layers. all f-layers will have the same number of nodes
    max_num_hidden = 257
    
    
    num_steps = [num_steps] * number_of_nets
    batch_size = np.random.randint(min_batch_size, max_batch_size, size=number_of_nets)
    patch_size = np.random.choice(patch_size, size=number_of_nets)
    layer = architecture[:-1]

    depth = []
    if layer and layer[0]=='c':
        layer = layer[1:]
        depth.append(np.random.randint(min_depth, max_depth, size=number_of_nets)) # 2 - 21
    if layer and layer[0]=='c':
        layer = layer[1:]
        depth.append(np.random.randint(min_depth, max_depth, size=number_of_nets) + depth[0])
    if layer and layer[0]=='c':
        layer = layer[1:]
        depth.append(np.random.randint(min_depth, max_depth, size=number_of_nets) + depth[1])
    if layer and layer[0]=='c':
        layer = layer[1:]
        depth.append(np.random.randint(min_depth, max_depth, size=number_of_nets) + depth[2])
    if layer and layer[0]=='c':
        layer = layer[1:]
        depth.append(np.random.randint(min_depth, max_depth, size=number_of_nets) + depth[3])
    if layer and layer[0]=='c':
        layer = layer[1:]
        depth.append(np.random.randint(min_depth, max_depth, size=number_of_nets) + depth[4])

    num_hidden = np.random.randint(min_num_hidden, max_num_hidden, size=number_of_nets)
    
    # Combining the hyperparameters to fit them into a for-loop
    hyperparameter = zip(num_steps, batch_size, patch_size, zip(*depth), num_hidden)
    
    return num_labels, num_channels, hyperparameter

In [8]:
def getSessConf(per_process_gpu_memory_fraction = 0.1, op_parallelism_threads = 18):
    gpu_config = tf.GPUOptions(allow_growth=True, per_process_gpu_memory_fraction=per_process_gpu_memory_fraction)
    session_conf = tf.ConfigProto(gpu_options=gpu_config, intra_op_parallelism_threads=op_parallelism_threads, inter_op_parallelism_threads=op_parallelism_threads)
    
    return session_conf

In [71]:
def initWeightsBiases():
    # Maximal 6 Convolution Layers & 5 Fully Connectd Layers
    tf.reset_default_graph()

    conv_size_dict={1:23*23, 2:12*12, 3:6*6, 4:3*3, 5:2*2, 6:1*1}
    with tf.Session(config=getSessConf()) as sess:
        weights_biases = []

        if len(depth)>=1:
            conv2d_1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth[0]], stddev=0.1), name='W')
            conv2d_1_biases = tf.Variable(tf.constant(1.0, shape=[depth[0]]), name='B')

        if len(depth)>=2:
            conv2d_2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth[0], depth[1]], stddev=0.1), name='W')
            conv2d_2_biases = tf.Variable(tf.constant(1.0, shape=[depth[1]]), name='B')

        if len(depth)>=3:
            conv2d_3_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth[1], depth[2]], stddev=0.1), name='W')
            conv2d_3_biases = tf.Variable(tf.constant(1.0, shape=[depth[2]]), name='B')

        if len(depth)>=4:
            conv2d_4_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth[2], depth[3]], stddev=0.1), name='W')
            conv2d_4_biases = tf.Variable(tf.constant(1.0, shape=[depth[3]]), name='B')

        if len(depth)>=5:
            conv2d_5_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth[3], depth[4]], stddev=0.1), name='W')
            conv2d_5_biases = tf.Variable(tf.constant(1.0, shape=[depth[4]]), name='B')

        if len(depth)>=6:
            conv2d_6_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth[4], depth[5]], stddev=0.1), name='W')
            conv2d_6_biases = tf.Variable(tf.constant(1.0, shape=[depth[5]]), name='B')

        shape = conv_size_dict[c_count]*depth[-1]

        if f_count-1>=1:
            fc_1_weights = tf.Variable(tf.truncated_normal([shape, num_hidden], stddev=0.1), name='W')
            fc_1_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]), name='B')
            shape = num_hidden

        if f_count-1>=2:
            fc_2_weights = tf.Variable(tf.truncated_normal([shape, num_hidden], stddev=0.1), name='W')
            fc_2_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]), name='B')
            shape = num_hidden

        if f_count-1>=3:
            fc_3_weights = tf.Variable(tf.truncated_normal([shape, num_hidden], stddev=0.1), name='W')
            fc_3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]), name='B')
            shape = num_hidden

        if f_count-1>=4:
            fc_4_weights = tf.Variable(tf.truncated_normal([shape, num_hidden], stddev=0.1), name='W')
            fc_4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]), name='B')
            shape = num_hidden

            
        # Output Layers
        if c_count>=1:
            shape = conv_size_dict[1]*depth[0]
            out_1_weights = tf.Variable(tf.truncated_normal([shape, num_labels], stddev=0.1), name='W')
            out_1_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='B')
        
        if c_count>=2:
            shape = conv_size_dict[2]*depth[1]
            out_2_weights = tf.Variable(tf.truncated_normal([shape, num_labels], stddev=0.1), name='W')
            out_2_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='B')
        
        if c_count>=3:
            shape = conv_size_dict[3]*depth[2]
            out_3_weights = tf.Variable(tf.truncated_normal([shape, num_labels], stddev=0.1), name='W')
            out_3_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='B')
        
        if c_count>=4:
            shape = conv_size_dict[4]*depth[3]
            out_4_weights = tf.Variable(tf.truncated_normal([shape, num_labels], stddev=0.1), name='W')
            out_4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='B')
        
        if c_count>=5:
            shape = conv_size_dict[5]*depth[4]
            out_5_weights = tf.Variable(tf.truncated_normal([shape, num_labels], stddev=0.1), name='W')
            out_5_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='B')
        
        if c_count>=6:
            shape = conv_size_dict[6]*depth[5]
            out_6_weights = tf.Variable(tf.truncated_normal([shape, num_labels], stddev=0.1), name='W')
            out_6_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='B')
        
        if f_count-1>=1:
            shape = num_hidden
            out_7_weights = tf.Variable(tf.truncated_normal([shape, num_labels], stddev=0.1), name='W')
            out_7_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='B')
        
        if f_count-1>=2:
            shape = num_hidden
            out_8_weights = tf.Variable(tf.truncated_normal([shape, num_labels], stddev=0.1), name='W')
            out_8_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='B')
        
        if f_count-1>=3:
            shape = num_hidden
            out_9_weights = tf.Variable(tf.truncated_normal([shape, num_labels], stddev=0.1), name='W')
            out_9_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='B')
        
        if f_count-1>=4:
            shape = num_hidden
            out_10_weights = tf.Variable(tf.truncated_normal([shape, num_labels], stddev=0.1), name='W')
            out_10_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='B')
        
        


        sess.run(tf.global_variables_initializer())

        if len(depth)>=1:
            weights_biases.append([conv2d_1_weights.eval(), conv2d_1_biases.eval()])
        if len(depth)>=2:
            weights_biases.append([conv2d_2_weights.eval(), conv2d_2_biases.eval()])
        if len(depth)>=3:
            weights_biases.append([conv2d_3_weights.eval(), conv2d_3_biases.eval()])
        if len(depth)>=4:
            weights_biases.append([conv2d_4_weights.eval(), conv2d_4_biases.eval()])
        if len(depth)>=5:
            weights_biases.append([conv2d_5_weights.eval(), conv2d_5_biases.eval()])
        if len(depth)>=6:
            weights_biases.append([conv2d_6_weights.eval(), conv2d_6_biases.eval()])

        if f_count-1>=1:
            weights_biases.append([fc_1_weights.eval(), fc_1_biases.eval()])
        if f_count-1>=2:
            weights_biases.append([fc_2_weights.eval(), fc_2_biases.eval()])
        if f_count-1>=3:
            weights_biases.append([fc_3_weights.eval(), fc_3_biases.eval()])
        if f_count-1>=4:
            weights_biases.append([fc_4_weights.eval(), fc_4_biases.eval()])
            
        if c_count>=1:
            weights_biases.append([out_1_weights.eval(), out_1_biases.eval()])
        if c_count>=2:
            weights_biases.append([out_2_weights.eval(), out_2_biases.eval()])
        if c_count>=3:
            weights_biases.append([out_3_weights.eval(), out_3_biases.eval()])
        if c_count>=4:
            weights_biases.append([out_4_weights.eval(), out_4_biases.eval()])
        if c_count>=5:
            weights_biases.append([out_5_weights.eval(), out_5_biases.eval()])
        if c_count>=6:
            weights_biases.append([out_6_weights.eval(), out_6_biases.eval()])
        if f_count-1>=1:
            weights_biases.append([out_7_weights.eval(), out_7_biases.eval()])
        if f_count-1>=2:
            weights_biases.append([out_8_weights.eval(), out_8_biases.eval()])
        if f_count-1>=3:
            weights_biases.append([out_9_weights.eval(), out_9_biases.eval()])
        if f_count-1>=4:
            weights_biases.append([out_01_weights.eval(), out_10_biases.eval()])
        

        #weights_biases.append([fc_5_weights.eval(), fc_5_biases.eval()])

    return weights_biases

In [72]:
# Input arguments from outside
#path_mc_images = sys.argv[1]
#save_model_path = sys.argv[2]

path_mc_images = '/fhgfs/users/jbehnken/make_Data/MC_diffuse_flat_preprocessed_images.h5'
save_model_path = '/fhgfs/users/jbehnken/crap'

In [74]:
# Number of events in training-dataset
num_events = 5000

# Number of events in validation-/test-dataset
events_in_validation_and_testing = 1000

# Number of nets to compute
number_of_nets = 1

# Comment on the run
title_name = 'test_test'

# Architectures to test
test_architectures = ['cf', 'cfff', 'ccf', 'ccfff', 'cccfff', 'cccccffff']



gamma_anteil, hadron_anteil, gamma_count, hadron_count = metaYielder(path_mc_images)
valid_dataset, valid_labels, test_dataset, test_labels = getValidationTesting(path_mc_images, events_in_validation_and_testing, gamma_anteil, hadron_anteil, gamma_count, hadron_count)


for architecture in test_architectures:
    c_count = architecture.count('c')
    f_count = architecture.count('f')
    folder_path = createFolders(architecture, save_model_path)
    print('\n\n', folder_path)
    
    best_auc = bestAuc(folder_path, architecture)
        
    num_labels, num_channels, hyperparameter = getHyperparameter(architecture, number_of_nets)

    for num_steps, batch_size, patch_size, depth, num_hidden in hyperparameter:
        print('\n', num_steps, batch_size, patch_size, depth, num_hidden)
        
        weights_biases = initWeightsBiases()
        print('Initialized')
        
        for pretraining_step in range(len(architecture)):
            print('Pretraining: {}'.format(architecture[:pretraining_step+1]))
            
            
            # Measuring the loop-time
            start = time.time()
            # Path to logfiles and correct file name
            LOGDIR = '/fhgfs/users/jbehnken/tf_logs/small_logs'
            # Getting the right count-number for the new logfiles
            logcount = str(len(os.listdir(LOGDIR)))
            hparams = '_bs={}_ps={}_d={}_nh={}_ns={}'.format(batch_size, patch_size, depth, num_hidden, num_steps)
            
            
            tf.reset_default_graph()
            with tf.Session(config=getSessConf()) as sess:
                # Create tf.variables for the three different datasets
                tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, 46, 45, num_channels), name='train_data')
                tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels), name='train_labels')

                tf_valid_dataset = tf.constant(valid_dataset, name='valid_data')
                tf_valid_labels = tf.constant(valid_labels, name='valid_labels')

                tf_test_dataset_final = tf.constant(test_dataset, name='test_data_final')
                tf_test_labels_final = tf.constant(test_labels, name='test_labels_final')                    

                # Summary for same example input images
                tf.summary.image('input', tf_train_dataset, 6)
                
                
                
                # Creating the graph. Only layers specified in 'model_name' will be added and correctly sized
                layer = architecture[:-1]

                if layer and layer[0]=='c' and pretraining_step>=0:
                    layer = layer[1:]
                    with tf.name_scope('conv2d_1'):
                        init_w_1 = tf.constant(weights_biases[0][0])
                        layer1_weights = tf.get_variable('W_1', initializer=init_w_1)
                        init_b_1 = tf.constant(weights_biases[0][1])
                        layer1_biases = tf.get_variable('B_1', initializer=init_b_1)

                        conv = tf.nn.conv2d(tf_train_dataset, layer1_weights, [1, 1, 1, 1], padding='SAME')
                        hidden = tf.nn.relu(conv + layer1_biases)
                        pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
                        pool = tf.nn.dropout(pool, 0.9)

                        tf.summary.histogram("weights", layer1_weights)
                        tf.summary.histogram("biases", layer1_biases)
                        tf.summary.histogram("activations", hidden)
                        tf.summary.histogram("pooling", pool)
                        
                        print('Conv-Layer 1 initialized')
                        
                        
                        
                if layer and layer[0]=='c' and pretraining_step>=1:
                    layer = layer[1:]
                    with tf.name_scope('conv2d_2'):
                        init_w_2 = tf.constant(weights_biases[1][0])
                        layer2_weights = tf.get_variable('W_2', initializer=init_w_2)
                        init_b_2 = tf.constant(weights_biases[1][1])
                        layer2_biases = tf.get_variable('B_2', initializer=init_b_2)

                        conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
                        hidden = tf.nn.relu(conv + layer2_biases)
                        pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
                        pool = tf.nn.dropout(pool, 0.9)

                        tf.summary.histogram("weights", layer2_weights)
                        tf.summary.histogram("biases", layer2_biases)
                        tf.summary.histogram("activations", hidden)
                        tf.summary.histogram("pooling", pool)
                        
                        print('Conv-Layer 2 initialized')
                        
                        
                        
                if layer and layer[0]=='c' and pretraining_step>=2:
                    layer = layer[1:]
                    with tf.name_scope('conv2d_3'):
                        init_w_3 = tf.constant(weights_biases[2][0])
                        layer3_weights = tf.get_variable('W_3', initializer=init_w_3)
                        init_b_3 = tf.constant(weights_biases[2][1])
                        layer3_biases = tf.get_variable('B_3', initializer=init_b_3)

                        conv = tf.nn.conv2d(pool, layer3_weights, [1, 1, 1, 1], padding='SAME')
                        hidden = tf.nn.relu(conv + layer3_biases)
                        pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
                        pool = tf.nn.dropout(pool, 0.9)

                        tf.summary.histogram("weights", layer3_weights)
                        tf.summary.histogram("biases", layer3_biases)
                        tf.summary.histogram("activations", hidden)
                        tf.summary.histogram("pooling", pool)
                        
                        print('Conv-Layer 3 initialized')
                        
                        
                        
                        
                if layer and layer[0]=='c' and pretraining_step>=3:
                    layer = layer[1:]
                    with tf.name_scope('conv2d_4'):
                        init_w_4 = tf.constant(weights_biases[3][0])
                        layer4_weights = tf.get_variable('W_4', initializer=init_w_4)
                        init_b_4 = tf.constant(weights_biases[3][1])
                        layer4_biases = tf.get_variable('B_4', initializer=init_b_4)

                        conv = tf.nn.conv2d(pool, layer4_weights, [1, 1, 1, 1], padding='SAME')
                        hidden = tf.nn.relu(conv + layer4_biases)
                        pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
                        pool = tf.nn.dropout(pool, 0.9)

                        tf.summary.histogram("weights", layer4_weights)
                        tf.summary.histogram("biases", layer4_biases)
                        tf.summary.histogram("activations", hidden)
                        tf.summary.histogram("pooling", pool)
                        
                        print('Conv-Layer 4 initialized')
                        
                        
                        
                if layer and layer[0]=='c' and pretraining_step>=4:
                    layer = layer[1:]
                    with tf.name_scope('conv2d_5'):
                        init_w_5 = tf.constant(weights_biases[4][0])
                        layer5_weights = tf.get_variable('W_5', initializer=init_w_5)
                        init_b_5 = tf.constant(weights_biases[4][1])
                        layer5_biases = tf.get_variable('B_5', initializer=init_b_5)

                        conv = tf.nn.conv2d(pool, layer5_weights, [1, 1, 1, 1], padding='SAME')
                        hidden = tf.nn.relu(conv + layer5_biases)
                        pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
                        pool = tf.nn.dropout(pool, 0.9)

                        tf.summary.histogram("weights", layer5_weights)
                        tf.summary.histogram("biases", layer5_biases)
                        tf.summary.histogram("activations", hidden)
                        tf.summary.histogram("pooling", pool)
                        
                        print('Conv-Layer 5 initialized')
                        
                        
                        
                if layer and layer[0]=='c' and pretraining_step>=5:
                    layer = layer[1:]
                    with tf.name_scope('conv2d_6'):
                        init_w_6 = tf.constant(weights_biases[5][0])
                        layer6_weights = tf.get_variable('W_6', initializer=init_w_6)
                        init_b_6 = tf.constant(weights_biases[5][1])
                        layer6_biases = tf.get_variable('B_6', initializer=init_b_6)

                        conv = tf.nn.conv2d(pool, layer6_weights, [1, 1, 1, 1], padding='SAME')
                        hidden = tf.nn.relu(conv + layer6_biases)
                        pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
                        pool = tf.nn.dropout(pool, 0.9)

                        tf.summary.histogram("weights", layer6_weights)
                        tf.summary.histogram("biases", layer6_biases)
                        tf.summary.histogram("activations", hidden)
                        tf.summary.histogram("pooling", pool)
                        
                        print('Conv-Layer 6 initialized')
                        
                        
                        
                # Reshape convolution layers to process the nodes further with connected layers
                with tf.name_scope('reshape'):
                    shape = pool.get_shape().as_list()
                    output = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
                    
                    
                    
                    
                    
                if layer and layer[0]=='f' and pretraining_step>=c_count:
                    layer = layer[1:]
                    with tf.name_scope('fc_7'):
                        init_w_7 = tf.constant(weights_biases[c_count][0])
                        layer7_weights = tf.get_variable('W_7', initializer=init_w_7)
                        init_b_7 = tf.constant(weights_biases[c_count][1])
                        layer7_biases = tf.get_variable('B_7', initializer=init_b_7)

                        hidden = tf.nn.relu(tf.matmul(output, layer7_weights) + layer7_biases)
                        output = tf.nn.dropout(hidden, 0.5)

                        tf.summary.histogram("weights", layer7_weights)
                        tf.summary.histogram("biases", layer7_biases)
                        tf.summary.histogram("activations", hidden)
                        
                        print('Fc-Layer 1 initialized')
                        
                        
                        
                if layer and layer[0]=='f' and pretraining_step>=c_count+1:
                    layer = layer[1:]
                    with tf.name_scope('fc_8'):
                        init_w_8 = tf.constant(weights_biases[c_count+1][0])
                        layer8_weights = tf.get_variable('W_8', initializer=init_w_8)
                        init_b_8 = tf.constant(weights_biases[c_count+1][1])
                        layer8_biases = tf.get_variable('B_8', initializer=init_b_8)

                        hidden = tf.nn.relu(tf.matmul(output, layer8_weights) + layer8_biases)
                        output = tf.nn.dropout(hidden, 0.5)

                        tf.summary.histogram("weights", layer8_weights)
                        tf.summary.histogram("biases", layer8_biases)
                        tf.summary.histogram("activations", hidden)
                        
                        print('Fc-Layer 2 initialized')
                        
                        
                        
                if layer and layer[0]=='f' and pretraining_step>=c_count+2:
                    layer = layer[1:]
                    with tf.name_scope('fc_9'):
                        init_w_9 = tf.constant(weights_biases[c_count+2][0])
                        layer9_weights = tf.get_variable('W_9', initializer=init_w_9)
                        init_b_9 = tf.constant(weights_biases[c_count+2][1])
                        layer9_biases = tf.get_variable('B_9', initializer=init_b_9)

                        hidden = tf.nn.relu(tf.matmul(output, layer9_weights) + layer9_biases)
                        output = tf.nn.dropout(hidden, 0.5)

                        tf.summary.histogram("weights", layer9_weights)
                        tf.summary.histogram("biases", layer9_biases)
                        tf.summary.histogram("activations", hidden)
                        
                        print('Fc-Layer 3 initialized')
                        
                        
                        
                if layer and layer[0]=='f' and pretraining_step>=c_count+3:
                    layer = layer[1:]
                    with tf.name_scope('fc_10'):
                        init_w_10 = tf.constant(weights_biases[c_count+3][0])
                        layer10_weights = tf.get_variable('W_10', initializer=init_w_10)
                        init_b_10 = tf.constant(weights_biases[c_count+3][1])
                        layer10_biases = tf.get_variable('B_10', initializer=init_b_10)

                        hidden = tf.nn.relu(tf.matmul(output, layer10_weights) + layer10_biases)
                        output = tf.nn.dropout(hidden, 0.5)

                        tf.summary.histogram("weights", layer10_weights)
                        tf.summary.histogram("biases", layer10_biases)
                        tf.summary.histogram("activations", hidden)
                        
                        print('Fc-Layer 4 initialized')
                        
                        
                with tf.name_scope('output_layer'):
                    i = len(architecture)-1-pretraining_step
                    
                    init_w_out = tf.constant(weights_biases[-i][0])
                    layerout_weights = tf.get_variable('W_out', initializer=init_w_out)
                    init_b_out = tf.constant(weights_biases[i][-1])
                    layerout_biases = tf.get_variable('B_out', initializer=init_b_out)

                    hidden = tf.nn.relu(tf.matmul(output, layerout_weights) + layerout_biases)
                    output = tf.nn.dropout(hidden, 0.5)

                    tf.summary.histogram("weights", layerout_weights)
                    tf.summary.histogram("biases", layerout_biases)
                    tf.summary.histogram("activations", hidden)
                        
                    print('Output-Layer initialized')



 /fhgfs/users/jbehnken/crap/1_cf

 20001 147 5 (4,) 212
Initialized
Pretraining: c
Conv-Layer 1 initialized
Output-Layer initialized
Pretraining: cf
Conv-Layer 1 initialized


ValueError: Shape must be rank 2 but is rank 4 for 'output_layer/MatMul' (op: 'MatMul') with input shapes: [147,2116], [5,5,1,4].

In [58]:
8993/529

17.0

In [24]:
23*23
12*12
6*6
3*3
2*2
1*1

1

In [27]:
conv_size_dict={1:23*23, 2:12*12, 3:6*6, 4:3*3, 5:2*2, 6:1*1}

In [21]:
print('Layer: {}'.format(len(weights_biases)))

print('Tensors: {}'.format(len(weights_biases[0])))

print('Tensor-Type: {}'.format(type(weights_biases[0][0])))

Layer: 4
Tensors: 2
Tensor-Type: <class 'numpy.ndarray'>


In [99]:
# Main loop with the training process
for num_steps, learning_rate, batch_size, patch_size, depth, num_hidden in hyperparameter:
    try:
        
        # Creating the graph. Only layers specified in 'model_name' will be added and correctly sized
        layer = model_name[:-1]
        
        if layer and layer[0]=='c':
            layer = layer[1:]
            with tf.name_scope('conv2d_1'):
                conv2d_1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth[0]], stddev=0.1), name='W')
                conv2d_1_biases = tf.Variable(tf.constant(1.0, shape=[depth[0]]), name='B')

                conv = tf.nn.conv2d(tf_train_dataset, conv2d_1_weights, [1, 1, 1, 1], padding='SAME')
                hidden = tf.nn.relu(conv + conv2d_1_biases)
                pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

                tf.summary.histogram("weights", conv2d_1_weights)
                tf.summary.histogram("biases", conv2d_1_biases)
                tf.summary.histogram("activations", hidden)
                tf.summary.histogram("pooling", pool)
    
    

        
        # Reshape convolution layers to process the nodes further with connected layers
        with tf.name_scope('reshape'):
            shape = pool.get_shape().as_list()
            output = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    
    

        if layer and layer[0]=='f':
            layer = layer[1:]        
            with tf.name_scope('fc_1'):
                shape = output.get_shape().as_list()
                fc_1_weights = tf.Variable(tf.truncated_normal([shape[1], num_hidden], stddev=0.1), name='W')
                fc_1_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]), name='B')

                output = tf.nn.relu(tf.matmul(output, fc_1_weights) + fc_1_biases)

                tf.summary.histogram("weights", fc_1_weights)
                tf.summary.histogram("biases", fc_1_biases)
                tf.summary.histogram("activations", output)
                
                

        if layer and layer[0]=='f':
            layer = layer[1:]        
            with tf.name_scope('fc_2'):
                shape = output.get_shape().as_list()
                fc_2_weights = tf.Variable(tf.truncated_normal([shape[1], num_hidden], stddev=0.1), name='W')
                fc_2_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]), name='B')

                output = tf.nn.relu(tf.matmul(output, fc_2_weights) + fc_2_biases)

                tf.summary.histogram("weights", fc_2_weights)
                tf.summary.histogram("biases", fc_2_biases)
                tf.summary.histogram("activations", output)
                
                

        if layer and layer[0]=='f':
            layer = layer[1:]        
            with tf.name_scope('fc_3'):
                shape = output.get_shape().as_list()
                fc_3_weights = tf.Variable(tf.truncated_normal([shape[1], num_hidden], stddev=0.1), name='W')
                fc_3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]), name='B')

                output = tf.nn.relu(tf.matmul(output, fc_3_weights) + fc_3_biases)

                tf.summary.histogram("weights", fc_3_weights)
                tf.summary.histogram("biases", fc_3_biases)
                tf.summary.histogram("activations", output)
                
                

        if layer and layer[0]=='f':
            layer = layer[1:]        
            with tf.name_scope('fc_4'):
                shape = output.get_shape().as_list()
                fc_4_weights = tf.Variable(tf.truncated_normal([shape[1], num_hidden], stddev=0.1), name='W')
                fc_4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]), name='B')

                output = tf.nn.relu(tf.matmul(output, fc_4_weights) + fc_4_biases)

                tf.summary.histogram("weights", fc_4_weights)
                tf.summary.histogram("biases", fc_4_biases)
                tf.summary.histogram("activations", output)
                
                

       
        with tf.name_scope('output'):
            shape = output.get_shape().as_list()
            output_weights = tf.Variable(tf.truncated_normal([shape[1], num_labels], stddev=0.1), name='W')
            output_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='B')

            output = tf.matmul(output, output_weights) + output_biases

            tf.summary.histogram("weights", output_weights)
            tf.summary.histogram("biases", output_biases)
            tf.summary.histogram("activations", output)
                
    
    
        # Computing the loss of the model
        with tf.name_scope('loss'):
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=tf_train_labels), name='loss')
            tf.summary.scalar("loss", loss)
    
        # Optimizing the model
        with tf.name_scope('optimizer'):
            optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)
    
        # Predictions for the training, validation, and test data
        with tf.name_scope('prediction'):
            train_prediction = tf.nn.softmax(output)
    
        with tf.name_scope('accuracy'):
            correct_prediction = tf.equal(tf.argmax(train_prediction, 1), tf.argmax(tf_train_labels, 1))
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
            tf.summary.scalar('batch_accuracy', accuracy)
                                                    
                
        
        # Computing the validation-dataset
        with tf.name_scope('validation'):
            layer = model_name[:-1]

            if layer and layer[0]=='c':
                layer = layer[1:]
                pool = tf.nn.max_pool(tf.nn.relu(tf.nn.conv2d(tf_valid_dataset, conv2d_1_weights, [1, 1, 1, 1], padding='SAME') + conv2d_1_biases), ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
            if layer and layer[0]=='c':
                layer = layer[1:]
                pool = tf.nn.max_pool(tf.nn.relu(tf.nn.conv2d(pool, conv2d_2_weights, [1, 1, 1, 1], padding='SAME')  + conv2d_2_biases), ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
            if layer and layer[0]=='c':
                layer = layer[1:]
                pool = tf.nn.max_pool(tf.nn.relu(tf.nn.conv2d(pool, conv2d_3_weights, [1, 1, 1, 1], padding='SAME')  + conv2d_3_biases), ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
            if layer and layer[0]=='c':
                layer = layer[1:]
                pool = tf.nn.max_pool(tf.nn.relu(tf.nn.conv2d(pool, conv2d_4_weights, [1, 1, 1, 1], padding='SAME')  + conv2d_4_biases), ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
            if layer and layer[0]=='c':
                layer = layer[1:]
                pool = tf.nn.max_pool(tf.nn.relu(tf.nn.conv2d(pool, conv2d_5_weights, [1, 1, 1, 1], padding='SAME')  + conv2d_5_biases), ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
            if layer and layer[0]=='c':
                layer = layer[1:]
                pool = tf.nn.max_pool(tf.nn.relu(tf.nn.conv2d(pool, conv2d_6_weights, [1, 1, 1, 1], padding='SAME')  + conv2d_6_biases), ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
            shape = pool.get_shape().as_list()
            output = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
            if layer and layer[0]=='f':
                layer = layer[1:]
                output = tf.nn.relu(tf.matmul(output, fc_1_weights) + fc_1_biases)
            if layer and layer[0]=='f':
                layer = layer[1:]
                output = tf.nn.relu(tf.matmul(output, fc_2_weights) + fc_2_biases)
            if layer and layer[0]=='f':
                layer = layer[1:]
                output = tf.nn.relu(tf.matmul(output, fc_3_weights) + fc_3_biases)
            if layer and layer[0]=='f':
                layer = layer[1:]
                output = tf.nn.relu(tf.matmul(output, fc_4_weights) + fc_4_biases)
            valid_prediction = tf.nn.softmax(tf.matmul(output, output_weights) + output_biases)
                                
            correct_prediction = tf.equal(tf.argmax(valid_prediction, 1), tf.argmax(valid_labels, 1))
            valid_accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
            tf.summary.scalar('validation_accuracy', valid_accuracy)                        
                                
        with tf.name_scope('auc'):
            valid_auc = tf.metrics.auc(labels=tf_valid_labels, predictions=valid_prediction, curve='ROC')
            tf.summary.scalar('validation_auc_0', valid_auc[0])
            #tf.summary.scalar('validation_auc_1', valid_auc[1])
            
            
            
        # Computing the test-dataset
        with tf.name_scope('testing'):
            layer = model_name[:-1]

            if layer and layer[0]=='c':
                layer = layer[1:]
                pool = tf.nn.max_pool(tf.nn.relu(tf.nn.conv2d(tf_test_dataset_final, conv2d_1_weights, [1, 1, 1, 1], padding='SAME') + conv2d_1_biases), ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
            if layer and layer[0]=='c':
                layer = layer[1:]
                pool = tf.nn.max_pool(tf.nn.relu(tf.nn.conv2d(pool, conv2d_2_weights, [1, 1, 1, 1], padding='SAME')  + conv2d_2_biases), ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
            if layer and layer[0]=='c':
                layer = layer[1:]
                pool = tf.nn.max_pool(tf.nn.relu(tf.nn.conv2d(pool, conv2d_3_weights, [1, 1, 1, 1], padding='SAME')  + conv2d_3_biases), ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
            if layer and layer[0]=='c':
                layer = layer[1:]
                pool = tf.nn.max_pool(tf.nn.relu(tf.nn.conv2d(pool, conv2d_4_weights, [1, 1, 1, 1], padding='SAME')  + conv2d_4_biases), ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
            if layer and layer[0]=='c':
                layer = layer[1:]
                pool = tf.nn.max_pool(tf.nn.relu(tf.nn.conv2d(pool, conv2d_5_weights, [1, 1, 1, 1], padding='SAME')  + conv2d_5_biases), ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
            if layer and layer[0]=='c':
                layer = layer[1:]
                pool = tf.nn.max_pool(tf.nn.relu(tf.nn.conv2d(pool, conv2d_6_weights, [1, 1, 1, 1], padding='SAME')  + conv2d_6_biases), ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
            shape = pool.get_shape().as_list()
            output = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
            if layer and layer[0]=='f':
                layer = layer[1:]
                output = tf.nn.relu(tf.matmul(output, fc_1_weights) + fc_1_biases)
            if layer and layer[0]=='f':
                layer = layer[1:]
                output = tf.nn.relu(tf.matmul(output, fc_2_weights) + fc_2_biases)
            if layer and layer[0]=='f':
                layer = layer[1:]
                output = tf.nn.relu(tf.matmul(output, fc_3_weights) + fc_3_biases)
            if layer and layer[0]=='f':
                layer = layer[1:]
                output = tf.nn.relu(tf.matmul(output, fc_4_weights) + fc_4_biases)
            test_prediction = tf.nn.softmax(tf.matmul(output, output_weights) + output_biases)
                                
            test_correct_prediction = tf.equal(tf.argmax(test_prediction, 1), tf.argmax(test_labels, 1))
            test_accuracy = tf.reduce_mean(tf.cast(test_correct_prediction, tf.float32))
            tf.summary.scalar('validation_accuracy', test_accuracy)                        
                                
        with tf.name_scope('auc'):
            test_auc = tf.metrics.auc(labels=tf_test_labels_final, predictions=test_prediction, curve='ROC')
            tf.summary.scalar('test_auc_0', test_auc[0])
            #tf.summary.scalar('validation_auc_1', test_auc[1])
            
    
        # Merge all summaries and create a saver
        summ = tf.summary.merge_all()
        saver = tf.train.Saver()
        
        # Initializing the model-variables and specify the logfiles
        sess.run(tf.global_variables_initializer())
        sess.run(tf.local_variables_initializer())
        writer = tf.summary.FileWriter(LOGDIR+'/'+logcount+hparams)
        writer.add_graph(sess.graph)
    
    
    
        # Iterating over num_steps batches and train the model 
        gen = batchYielder()
        for step in range(num_steps):
            batch_data, batch_labels = next(gen)
            # Creating a feed_dict to train the model on in this step
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            # Train the model for this step
            _, l, predictions = sess.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    
    
            # Updating the output to stay in touch with the training process
            # Checking for early-stopping with scikit-learn
            if (step % 100 == 0):
                s = sess.run(summ, feed_dict={tf_train_dataset: batch_data, tf_train_labels: batch_labels})
                writer.add_summary(s, step)
                
                # Compute the accuracy and the roc-auc-score with scikit-learn
                pred = sess.run(valid_prediction)
                pred = np.array(list(zip(pred[:,0], pred[:,1])))
                stop_acc = accuracy_score(np.argmax(valid_labels, axis=1), np.argmax(pred, axis=1))
                stop_auc = roc_auc_score(valid_labels, pred)
                      
                  
                # Check if early-stopping is necessary
                auc_now = stop_auc
                if step == 0:
                    stopping_auc = 0.0
                    sink_count = 0
                else:
                    if auc_now > stopping_auc:
                        stopping_auc = auc_now
                        sink_count = 0
                        # Check if the model is better than the existing one and has to be saved
                        if stopping_auc > best_auc:
                            saver.save(sess, os.path.join(folder_path, model_name))
                            best_auc = stopping_auc
                    else:
                        sink_count += 1
                        
                # Printing a current evaluation of the model
                print('St_auc: {}, sc: {},val: {}, Step: {}'.format(stopping_auc, sink_count, stop_acc*100, step))
                if sink_count == 10:
                    break   
    
        
        # Compute the final score of the model       
        pred = sess.run(test_prediction)
        pred = np.array(list(zip(pred[:,0], pred[:,1])))
        f_acc = accuracy_score(np.argmax(test_labels, axis=1), np.argmax(pred, axis=1))
        f_auc = roc_auc_score(test_labels, pred)
        
        # Close the session
        sess.close()
        
        print('Final_auc: {}, Final_acc: {}'.format(f_auc, f_acc))
        # Save the run to the csv and restart the loop
        dauer = time.time() - start
        early_stopped = True if step < num_steps-1 else False
        with open(os.path.join(folder_path, model_name+'_Hyperparameter.csv'), 'a') as f:
            writer = csv.writer(f)
            writer.writerow([learning_rate, batch_size, patch_size, *depth, num_hidden, f_acc*100, f_auc, step, early_stopped, dauer, title_name])

    except:
        sess.close()
        raise
